<a href="https://colab.research.google.com/github/ftrotter/SQLClass/blob/main/Fake_Sales_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker
!pip install pymysql
import sqlalchemy
import pandas as pd
import getpass
import faker
import random
from datetime import datetime

from faker import Faker
fake = Faker()

the_year = 2021

sales_people_count = 2 
avg_calls_per_day = 15

passwd = getpass.getpass('  enter the cardremote db password:')
hostname = getpass.getpass('  enter the hostname:')

engine = sqlalchemy.create_engine('mysql+pymysql://collab:' + passwd + '@' + hostname +'.set.care:3306/selling')



# given a day of the week, randomly determine how many calls were made that day
def random_sales_this_day_of_week(day_of_week_num):
  if day_of_week_num == 3 or day_of_week_num == 4:
    return(0)
  else: 
    return(random.randrange(10,25))


# Given a number of calls per day, randomly determine how many of them should succeed.
def random_successful_calls(total_calls):
  return(round((total_calls * random.randrange(0,100))/100))


# Given a day of the week... randomly decide if a given call worked
def did_this_call_work(day_of_week):

  threshold = 40
  if day_of_week == 3 or day_of_week == 4:
    threshold = 10
  chance = random.randrange(0,100)
  if chance > threshold: 
    return(True)
  else:
    return(False)



#Loop over all of the sales people
while sales_people_count > 0:

  sales_person_name = fake.name() 
  print(sales_person_name)

  year_day = 365


  # Save the given sales person here... 
  save_salesperson_sql = f"""
INSERT INTO selling.saleperson 
  (`id`, `salesperson_name`, `created_at`, `updated_at`) 
VALUES 
  (NULL, '{sales_person_name}', current_timestamp(), current_timestamp());

"""

  lastresult = engine.execute(save_salesperson_sql)
  salesperson_id = lastresult.lastrowid

  # And retain the id of the record for later use.

  # Loop over all of the days in a year... 
  while year_day > 0: 
    # = datetime.date(the_year, 1, 1) + datetime.timedelta(year_day - 1)
    d = datetime.strptime(str(the_year) + "-" + str(year_day), "%Y-%j")




    day_of_the_week = d.weekday()

    year_day = year_day - 1

    sales_calls_today = random_sales_this_day_of_week(day_of_the_week)
 
    # loop over every individual sales call
    while sales_calls_today > 0:
      sales_calls_today = sales_calls_today - 1
      if(did_this_call_work(day_of_the_week)):
        is_call_worked = 1
      else:
        is_call_worked = 0

      hour = random.randrange(7,19)
      min = random.randrange(1,60)
      seconds = random.randrange(1,60)

      date_for_mysql = d.strftime('%Y-%m-%d') + f" {hour}:{min}:{seconds}"

      insert_call_sql = f"""

INSERT INTO selling.callevent 
  (`id`, `salesperson_id`, `is_call_success`, `call_timedate`)
VALUES 
  (NULL, '{salesperson_id}', '{is_call_worked}', '{date_for_mysql}.000000');

"""
      lastresult = engine.execute(insert_call_sql)
      #print(f"person: {sales_person_name} date: {date_for_mysql} is_call_worked: {is_call_worked}  ")


  sales_people_count = sales_people_count - 1





Russell Boyle
Dustin Ortega
